In [ ]:
# 필요한 라이브러리 임포트
!pip install transformers torch datasets
!pip install transformers[torch]
!pip install accelerate>=0.26.0


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments
from datasets import Dataset

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                     bos_token='</s>', 
                                                     eos_token='</s>', 
                                                     pad_token='<pad>')
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2").to(device)

# 데이터 로드 및 전처리 (샘플링 포함)
def load_and_prepare_data(file_path, sample_size=1000):
    """
    주어진 데이터 파일을 불러와 모델 학습에 사용할 수 있도록 처리하며,
    sample_size 만큼의 데이터만 샘플링합니다.
    """
    data = pd.read_csv(file_path)  # CSV 파일 읽기
    data = data.sample(n=sample_size, random_state=42)  # 데이터 샘플링
    data['text'] = data.apply(
        lambda row: f"질문: {row['발화문']}" if row['QA여부'] == 'q' else f"답변: {row['발화문']}", axis=1
    )
    dataset = Dataset.from_pandas(data[['text']])
    return dataset

# 데이터 증강: 문맥적 삽입
def augment_data(dataset, context_insert="소상공인 고객 문의"):
    """
    데이터에 문맥을 삽입하여 증강합니다.
    """
    def augment(example):
        example['text'] = context_insert + " - " + example['text']
        return example
    return dataset.map(augment)

# 토크나이저로 텍스트 토큰화
def tokenize_function(examples):
    """
    텍스트를 토큰화하고 labels를 추가합니다.
    """
    tokenized = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    tokenized['labels'] = tokenized['input_ids'].copy()
    
    # 패딩 토큰 손실 무시
    padding_index = tokenizer.pad_token_id
    tokenized['labels'] = [
        [-100 if token == padding_index else token for token in label] for label in tokenized['labels']
    ]
    return tokenized

# 데이터셋 준비
file_path = "건강_train.csv"  # 사용자 데이터 경로
raw_dataset = load_and_prepare_data(file_path, sample_size=1000)  # 1,000개의 데이터만 샘플링
augmented_dataset = augment_data(raw_dataset)
tokenized_dataset = augmented_dataset.map(tokenize_function, batched=True)

# 데이터 분리
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# 훈련 설정
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Trainer 객체 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# 모델 학습
trainer.train()

# Perplexity 계산
def calculate_perplexity(eval_results):
    """
    Perplexity를 계산합니다.
    """
    loss = eval_results['eval_loss']
    return np.exp(loss)

# 평가 및 결과 출력
eval_results = trainer.evaluate()
perplexity = calculate_perplexity(eval_results)
print(f"Perplexity: {perplexity}")
